In [11]:
from mesa.space import MultiGrid
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random

def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum( xi * (N-i) for i,xi in enumerate(x) ) / (N*sum(x))
    return (1 + (1/N) - 2*B)

# model.py
class MoneyModel(Model):
    """A model with some number of agents."""
    def __init__(self, N, width, height):
        self.running = True
        self.num_agents = N
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = DataCollector(
            model_reporters={"Gini": compute_gini},
            agent_reporters={"Wealth": lambda a: a.wealth})

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


In [4]:
class MoneyAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other = random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()

In [12]:
# run.py
from mesa.batchrunner import BatchRunner

params = {"width": 10,
                "height": 10, 
                "N": range(10, 500, 10)}

batch_run = BatchRunner(MoneyModel,
                        parameter_values=params,
                        iterations=5,
                        max_steps=100,
                        model_reporters={"Gini": compute_gini})
batch_run.run_all()




  0%|          | 0/245 [00:00<?, ?it/s]

  2%|▏         | 5/245 [00:00<00:05, 45.03it/s]

  3%|▎         | 8/245 [00:00<00:06, 36.62it/s]

  4%|▍         | 11/245 [00:00<00:07, 31.18it/s]

  5%|▌         | 13/245 [00:00<00:09, 25.67it/s]

  6%|▌         | 15/245 [00:00<00:10, 22.76it/s]

  7%|▋         | 17/245 [00:00<00:12, 18.87it/s]

  8%|▊         | 19/245 [00:00<00:13, 16.80it/s]

  9%|▊         | 21/245 [00:01<00:14, 14.98it/s]

  9%|▉         | 23/245 [00:01<00:17, 12.83it/s]

 10%|█         | 25/245 [00:01<00:18, 12.18it/s]

 11%|█         | 27/245 [00:01<00:19, 11.15it/s]

 12%|█▏        | 29/245 [00:01<00:20, 10.48it/s]

 13%|█▎        | 31/245 [00:02<00:21,  9.78it/s]

 13%|█▎        | 33/245 [00:02<00:23,  9.15it/s]

 14%|█▍        | 34/245 [00:02<00:24,  8.71it/s]

 14%|█▍        | 35/245 [00:02<00:24,  8.41it/s]

 15%|█▍        | 36/245 [00:02<00:26,  7.97it/s]

 15%|█▌        | 37/245 [00:02<00:27,  7.69it/s]

 16%|█▌        | 38/245 [00:03<00:27,  7.52it/s]

 16%|█▌  

KeyboardInterrupt: 